# Day 2 - Lab 1: AI-Generated System Design & Database Seeding

**Objective:** Use the PRD artifact from Day 1 to generate a detailed SQL database schema, create realistic seed data, and then use those outputs to create and seed a live, local database file.

**Estimated Time:** 150 minutes

**Introduction:**
Welcome to Day 2! Today, we transition from *what* we're building to *how* we'll build it. In this lab, you will act as the lead architect for the Onboarding Tool. Your task is to use the PRD to define the data structure of the application and create a tangible database artifact that will be used for the rest of the course.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the `day1_prd.md` artifact from Day 1. This document is the primary source of truth for our project and provides the necessary context for the LLM to make intelligent design suggestions.

**Model Selection:**
Feel free to experiment with different models by changing the `model_name` in `setup_llm_client()`. Models with strong reasoning capabilities, like `gpt-4o`, `o3`, or `gemini-2.5-pro`, are excellent choices for design tasks.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the PRD from the `artifacts` directory.
- `save_artifact()`: To save the generated SQL schema and seed data.
- `clean_llm_output()`: To remove markdown fences from the generated SQL code.

In [ ]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-pro")

# Load the PRD from Day 1
prd_content = load_artifact("artifacts/day1_prd.md")
if not prd_content:
    print("Warning: Could not load day1_prd.md. Lab may not function correctly.")

## Step 2: The Challenges

### Challenge 1 (Foundational): Generating the SQL Schema

**Task:** Use the PRD to generate a normalized SQL schema for the application.

**Instructions:**
1.  Create a prompt that instructs the LLM to act as a Database Administrator (DBA).
2.  Provide the `prd_content` as context.
3.  Ask the LLM to design a normalized SQL schema with at least two tables (e.g., `users` and `onboarding_tasks`).
4.  The output should be the raw `CREATE TABLE` statements.
5.  Save the generated SQL to `artifacts/schema.sql`.

In [2]:
# TODO: Write a prompt to generate the SQL schema from the PRD.
schema_prompt = f"""
You are a database administrator and SQL expert. Based on the following Product Requirements Document (PRD), generate a complete SQL schema for a relational database. Ensure that the schema includes tables, columns with appropriate data types, primary keys, foreign keys, and any necessary constraints.
PRD:
{prd_content}
The output should be the raw CREATE TABLE statements without any additional explanations or comments.
"""
enhanced_schema_prompt = prompt_enhancer(schema_prompt)

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(enhanced_schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

2025-10-28 12:24:03,368 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


--- Generating SQL Schema ---
CREATE TABLE companies (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    name VARCHAR(255) NOT NULL,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    updated_at TIMESTAMPTZ NOT NULL DEFAULT NOW()
);

CREATE TABLE users (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    company_id UUID NOT NULL REFERENCES companies(id) ON DELETE RESTRICT,
    full_name VARCHAR(255) NOT NULL,
    email VARCHAR(255) NOT NULL,
    password_hash TEXT NOT NULL,
    role VARCHAR(50) NOT NULL CHECK (role IN ('hr_admin', 'hiring_manager', 'new_hire')),
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    updated_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    UNIQUE (company_id, email)
);

CREATE TABLE onboarding_plan_templates (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    company_id UUID NOT NULL REFERENCES companies(id) ON DELETE CASCADE,
    name VARCHAR(255) NOT NULL,
    description TEXT,
    created_at TIMESTAMPTZ NOT NULL DEFAULT NOW(),
    updated

In [ ]:
import sys
import os
import sqlite3

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output, prompt_enhancer

def generate_schema_with_model(prd_content, model_name):
    """Generate SQL schema using specified model"""
    client, model, provider = setup_llm_client(model_name=model_name)
    
    schema_prompt = f"""
    You are a database administrator and SQL expert. Based on the following Product Requirements Document (PRD), 
    generate a complete SQL schema for a relational database. The schema must include:
    1. All necessary tables for an onboarding system
    2. Appropriate columns with specific data types
    3. Primary and foreign key relationships
    4. Relevant constraints and indices
    5. Timestamp fields for audit purposes

    PRD:
    {prd_content}

    Provide only the raw CREATE TABLE statements without explanations.
    """
    
    enhanced_prompt = prompt_enhancer(schema_prompt)
    generated_schema = get_completion(enhanced_prompt, client, model, provider)
    return clean_llm_output(generated_schema, language='sql')

def compare_schemas():
    # Load the PRD
    prd_content = load_artifact("artifacts/day1_prd.md")
    if not prd_content:
        print("Error: Could not load PRD content")
        return

    # Models to compare
    models = [
        "gemini-2.5-pro",  # Google's model
        "claude-sonnet-4-20250514",     # Anthropic's model
        "gpt-5-2025-08-07",              # OpenAI's model
    ]

    # Generate schemas with each model
    for model in models:
        print(f"\n=== Generating schema with {model} ===\n")
        try:
            schema = generate_schema_with_model(prd_content, model)
            
            # Save to model-specific files
            filename = f"artifacts/schema_{model.replace('-', '_')}.sql"
            save_artifact(schema, filename)
            
            print(f"Schema generated with {model}:")
            print(schema)
            print("\n" + "="*50)
            
        except Exception as e:
            print(f"Error generating schema with {model}: {str(e)}")

if __name__ == "__main__":
    compare_schemas()


=== Generating schema with gemini-2.5-pro ===



2025-10-28 15:17:31,987 ag_aisoftdev.utils INFO LLM Client configured provider=google model=gemini-2.5-pro latency_ms=None artifacts_path=None
2025-10-28 15:17:32,737 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None
2025-10-28 15:19:23,890 ag_aisoftdev.utils ERROR Model 'gpt-4' is not in the list of recommended models. provider=None model=gpt-4 latency_ms=None artifacts_path=None
2025-10-28 15:19:23,900 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Error generating schema with gemini-2.5-pro: [google:gemini-2.5-pro] completion error: Server disconnected without sending a response.

=== Generating schema with gpt-4 ===



2025-10-28 15:19:41,811 ag_aisoftdev.utils ERROR Model 'claude-3' is not in the list of recommended models. provider=None model=claude-3 latency_ms=None artifacts_path=None
2025-10-28 15:19:41,822 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


Error generating schema with gpt-4: [None:None] completion error: API client not initialized.

=== Generating schema with claude-3 ===

Error generating schema with claude-3: [None:None] completion error: API client not initialized.


### Challenge 2 (Intermediate): Generating Realistic Seed Data

**Task:** Prompt the LLM to generate realistic seed data that conforms to the schema you just created.

**Instructions:**
1.  Create a new prompt.
2.  Provide both the `prd_content` and the `cleaned_schema` as context.
3.  Instruct the LLM to generate 5-10 realistic `INSERT` statements for your tables.
4.  The data should be relevant to a new hire onboarding tool (e.g., sample user names and task titles like "Complete HR Paperwork").
5.  Save the generated `INSERT` statements to `artifacts/seed_data.sql`.

In [3]:
# TODO: Write a prompt to generate realistic seed data.
seed_data_prompt = f"""
Using the following SQL schema, and Product Requirements Document, generate realistic SQL INSERT statements to seed the database with sample data. Ensure that the data respects all constraints defined in the schema, including primary keys, foreign keys, and data types.
SQL Schema:
{cleaned_schema}
Product Requirements Document (PRD):
{prd_content}
The output should be a series of 10 SQL INSERT statements without any additional explanations or comments.
The data should be relevant to the new hire onboarding tool application context described in the PRD.
"""

print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'artifacts/seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

--- Generating Seed Data ---
INSERT INTO companies (id, name) VALUES ('4d3b8f2d-8b1e-4a6c-9e2a-1b9d7f3c5e8a', 'Innovate Solutions Inc.');
INSERT INTO users (id, company_id, full_name, email, password_hash, role) VALUES ('a8b9c0d1-e2f3-4a5b-8c9d-0e1f2a3b4c5d', '4d3b8f2d-8b1e-4a6c-9e2a-1b9d7f3c5e8a', 'Hannah Reed', 'h.reed@innovatesolutions.com', '$2b$12$D8c.Q3.y9yZ8Q.c8N.f9uOF0Z.eJ5lZ3K.z6B.wX9tC.gH4iJ2j5O', 'hr_admin');
INSERT INTO users (id, company_id, full_name, email, password_hash, role) VALUES ('b1c2d3e4-f5a6-7b8c-9d0e-1f2a3b4c5d6e', '4d3b8f2d-8b1e-4a6c-9e2a-1b9d7f3c5e8a', 'Sarah Jones', 's.jones@innovatesolutions.com', '$2b$12$E9f.R4.z0aX7Q.d9O.g0vPG1Z.fK6mZ4L.a7C.xY0uD.hI5jK3k6P', 'hiring_manager');
INSERT INTO users (id, company_id, full_name, email, password_hash, role) VALUES ('c4d5e6f7-a8b9-0c1d-2e3f-4a5b6c7d8e9f', '4d3b8f2d-8b1e-4a6c-9e2a-1b9d7f3c5e8a', 'Priya Patel', 'p.patel@innovatesolutions.com', '$2b$12$F0g.S5.a1bY8R.e0P.h1wQH2A.gL7nZ5M.b8D.yZ1vE.iJ6lL4l7Q', 'new_hire

### Challenge 3 (Advanced): Creating and Seeding a Live Database

**Task:** This is a critical technical step. You will write a Python script to execute the generated SQL files, creating a live `onboarding.db` file that your application will use.

**Instructions:**
1.  Complete the `create_database` function below.
2.  The function should first connect to (and thus create) a SQLite database file named `artifacts/onboarding.db`.
3.  It should then open and execute the `schema.sql` file to create the tables.
4.  Finally, it should open and execute the `seed_data.sql` file to populate the tables.
5.  Use a `try...finally` block to ensure the database connection is always closed, even if an error occurs.

> **Hint:** The `try...finally` block is a crucial Python pattern. The code in the `finally` block will run whether the `try` block succeeds or fails, making it the perfect place to ensure resources like database connections are always closed.

**Expected Quality:** A physical `onboarding.db` file in your `artifacts` folder. This is a tangible asset that proves your design is valid and provides a concrete foundation for backend development.

In [ ]:
def create_database(db_path, schema_path, seed_path):
    """Creates and seeds a SQLite database from SQL files."""
    if not os.path.exists(schema_path):
        print(f"Error: Schema file not found at {schema_path}")
        return
    
    conn = None
    try:
        # TODO: Connect to the SQLite database. This will create the file if it doesn't exist.
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print(f"Successfully connected to database at {db_path}")

        # TODO: Read the content of the schema file using load_artifact.
        schema_sql = ""
        
        # TODO: Execute the schema SQL script.
        # Hint: Use cursor.executescript() for multi-statement SQL strings.
        
        print("Tables created successfully.")

        # TODO: Check if the seed data file exists. If it does, load and execute it.
        if os.path.exists(seed_path):
            # Load and execute the seed data
            pass # Your code here

        # TODO: Commit the changes to the database.
        
    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        # TODO: Ensure the connection is closed if it was opened.
        pass # Your code here

# Define file paths
db_file = os.path.join(project_root, "artifacts", "onboarding.db")
schema_file = os.path.join(project_root, "artifacts", "schema.sql")
seed_file = os.path.join(project_root, "artifacts", "seed_data.sql")

# Execute the function
create_database(db_file, schema_file, seed_file)

## Lab Conclusion

Excellent work! You have now moved from abstract requirements to a concrete, physical database artifact. You've used an LLM to design a schema, generate realistic test data, and then used a Python script to bring that database to life. This `onboarding.db` file is the foundation upon which we will build our API in Day 3.

> **Key Takeaway:** The ability to generate structured data definitions (like a SQL schema) from unstructured text (like a PRD) is a core skill in AI-assisted development. It automates a critical and often time-consuming design step.